In [2]:
setwd("/projects/PPC/analysis/ppc_eqtls")

source("scripts/packages.R"  )
source("scripts/functions.R" )
source("scripts/input_data.R")

library(coloc)

set.seed(5366)

Warning message:
“The `size` argument of `element_line()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”
Warning message:
“The `size` argument of `element_rect()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”
This is a new update to coloc.



# Run LD between PPC and Islet & Pancreas eQTLs

In [3]:
load("manuscript/robj/egene_list.robj", verbose = T)
names(egene_list)

load("manuscript/robj/gt1_list.robj", verbose = T)
names(gt1_list)

load("manuscript/robj/finemap_dir.robj", verbose = T)
names(finemap_dir)

gt1_list[["gene"]][["Islets"]]$snp = gt1_list[["gene"]][["Islets"]]$id
gt1_list[["gene"]][["iPSC-PPC"]]$snp = gt1_list[["gene"]][["iPSC-PPC"]]$id

Loading objects:
  egene_list


[1] "gene"    "isoform"

Loading objects:
  gt1_list


[1] "gene"    "isoform"

Loading objects:
  finemap_dir


[1] "gene"    "isoform"

In [4]:
geneinfo = fread("/reference/private/Gencode.v34lift37/gene_info.txt", data.table = F)  %>% 
    mutate(transcript_id = gene_id) %>% select(gene_id, transcript_id, chrom, start, end)

isofinfo = fread("/reference/private/Gencode.v34lift37/isoform_info.txt", data.table = F) %>% 
    select(gene_id, transcript_id, chrom, start, end)

geneinfo = rbind(geneinfo, isofinfo) %>% mutate(transcript_id = simplify_id(transcript_id))

# old

In [8]:
# analysis = "isoform"
# tissue = "Pancreas"
# taskid = 100

In [7]:
# out = data.frame(snp = unique(all_cred[all_cred$qtl_id == qtlid,]$snp))

# for (Tiss in c("iPSC-PPC", "Islets", "Pancreas"))
# {
#     out[,Tiss] = unlist(lapply(out$snp, function(x)
#     {
#         t = ld[ld$Var1 == x,]
        
#         if (nrow(t) == 0)
#         {
#             return(NA)
#         } else
#         {
#             t = ld[ld$Var1 == x & ld$Var2 %in% all_cred[all_cred$tiss == "iPSC-PPC" & all_cred$qtl_id != qtlid,]$snp ,]
#             return(max(t$value))
#         }
#     }))
# }

# colnames(out)[which(colnames(out) == "iPSC-PPC")] = "iPSC_PPC"
                                                   
# fwrite(out, paste0(outfile, ".ld.sum"), row.names = F, sep = "\t")

# message(paste("Saved:", paste0(outfile, ".ld.sum")))

# Plink per chromosome per tissue

In [6]:
# snp_per_chr = as.data.frame(rbindlist(lapply(names(gt1_list[["gene"]]), function(n)
# {
#     out = rbind(gt1_list[["gene"]][[n]] %>% select(snp), gt1_list[["isoform"]][[n]] %>% select(snp))
#     out$snp = gsub("chr", "", gsub("VAR_", "", out$snp))
#     out$chr = unlist(lapply(out$snp, function(x) { unlist(strsplit(x, "_"))[1] }))
#     out$tiss = n
#     out = out %>% filter(chr != "null")
#     return(out)
# })))

# table(snp_per_chr$chr, snp_per_chr$tiss)

In [5]:
# for (c in c(1:22)[1:9])
# {
#     for (Tiss1 in c("iPSC-PPC", "Islets", "Pancreas"))
#     {
#         for (Tiss2 in c("iPSC-PPC", "Islets", "Pancreas"))
#         {
#             # Get snps in chromosome in tissue 1
#             snps1 = unique(snp_per_chr[snp_per_chr$chr == c & snp_per_chr$tiss == Tiss1,]$snp)
            
#             # Get snps in chromosome in tissue 2
#             snps2 = unique(snp_per_chr[snp_per_chr$chr == c & snp_per_chr$tiss == Tiss2,]$snp)
            
#             # write snps input
#             snps = unique(c(snps1, snps2))
#             snpfile = paste("pipeline/17.eQTLFuncEnr/ld_all/input", paste(Tiss1, Tiss2, paste0("chr", c), "txt", sep = "."), sep = "/")
#             writeLines(snps, snpfile, sep = "\n")

#             # run plink
#             outfile = paste("pipeline/17.eQTLFuncEnr/ld_all/out", paste(Tiss1, Tiss2, paste0("chr", c), sep = "."), sep = "/")

#             cmd = paste("plink", "--memory 15000 --threads 4", "--vcf", paste0("~/references/1kg/vcf/", paste0("chr", c), ".vcf.gz"), 
#                   "--extract", snpfile, "--keep-allele-order", "--r2", "square", "--make-bed", "--out", outfile)

#             qsub_cmd = paste("echo", paste0("\"", cmd, "\"" ), "|", "qsub -N", paste(Tiss1, Tiss2, c, sep = "_"),
#                              "-pe smp 4", "-V -cwd", "-o", paste("logs", paste(Tiss1, Tiss2, c, "out", sep = "."), sep = "/"),
#                              "-l hostname=\"fl-n-1-17|fl-n-1-13|fl-n-1-9\"", 
#                              "-e", paste("logs", paste(Tiss1, Tiss2, c, "err", sep = "."), sep = "/"))
#             system(qsub_cmd)
#             message(qsub_cmd)
#         }   
#     }  
# }


# Summarize pairwise

In [147]:
save(gt1_list, file = "manuscript/robj/gt1_list.robj")

In [462]:
all_cred = as.data.frame(rbindlist(lapply(names(gt1_list[["gene"]]), function(name)
{
    as.data.frame(rbindlist(lapply(c("gene", "isoform"), function(analysis)
    {
        message(paste(name, analysis))
        if (name == "Pancreas" & analysis == "isoform")
        {
            out = gt1_list[[analysis]][[name]] %>% select(snp, qtl_id, transcript_id, gene_id) %>% distinct() %>%
                    mutate(snp = gsub("chr", "", gsub("VAR_", "", snp)), 
                           chr = unlist(lapply(snp, function(x) { unlist(strsplit(x, "_"))[1] }))) %>% 
                    mutate(tiss = name, phenotype = analysis) 
            out$gene_id = unlist(lapply(out$transcript_id, function(x) { simplify_id(unlist(strsplit(x, ":"))[5]) }))
            return(out)
        } else if (name == "Islets" & analysis == "isoform")
        {
            out = gt1_list[[analysis]][[name]] %>% select(snp, qtl_id, transcript_id, gene_id) %>% distinct() %>%
                    mutate(snp = gsub("chr", "", gsub("VAR_", "", snp)), 
                           chr = unlist(lapply(snp, function(x) { unlist(strsplit(x, "_"))[1] }))) %>% 
                    mutate(tiss = name, phenotype = analysis) 
            out$gene_id = simplify_id(out$transcript_id)
            return(out)
        } else
        {
            message(name)
            out = gt1_list[[analysis]][[name]] %>% select(snp, qtl_id, transcript_id, gene_id) %>% distinct() %>%
                    mutate(snp = gsub("chr", "", gsub("VAR_", "", snp)), 
                           chr = unlist(lapply(snp, function(x) { unlist(strsplit(x, "_"))[1] }))) %>% 
                    mutate(tiss = name, phenotype = analysis) 
            
            if (name == "iPSC-PPC" & analysis == "isoform")
            {
                out$gene_id = suppressMessages(mapvalues(out$transcript_id, from = geneinfo$transcript_id, to = simplify_id(geneinfo$gene_id)))
            }
            return(out)
        }
    })))
})))

fwrite(all_cred, "pipeline/17.eQTLFuncEnr/ld_all/summary/all_cred_sets_gt1.txt", row.names = F, sep = "\t")

iPSC-PPC gene

iPSC-PPC

iPSC-PPC isoform

iPSC-PPC

Islets gene

Islets

Islets isoform

Pancreas gene

Pancreas

Pancreas isoform



## Combine LD output

In [11]:
# files = list.files("pipeline/17.eQTLFuncEnr/ld_all/out")
# pairs = as.data.frame(rbindlist(lapply(files, function(f)
# {
#     tiss1 = unlist(strsplit(f, "[.]"))[1]
#     tiss2 = unlist(strsplit(f, "[.]"))[2]
#     chr = unlist(strsplit(f, "[.]"))[3]
#     data.frame(tiss1 = tiss1, tiss2 = tiss2, chr = chr)
# }))) %>% distinct()

# tiss1 = "iPSC-PPC"
# tiss2 = "iPSC-PPC"
# chr = 22

In [10]:
# ld = fread(paste("pipeline/17.eQTLFuncEnr/ld_all/out", paste(tiss1, tiss2, paste0("chr", chr), "ld", sep = "."), sep = "/"), data.table = F)
# bim = fread(paste("pipeline/17.eQTLFuncEnr/ld_all/out", paste(tiss1, tiss2, paste0("chr", chr), "bim", sep = "."), sep = "/"), data.table = F)

# rownames(ld) = bim$V2
# colnames(ld) = bim$V2

# a = suppressWarnings(melt(as.matrix(ld))) %>% filter(value > 0.6) %>% mutate(Var1 = as.vector(Var1), Var2 = as.vector(Var2)) 
# a$pos1 = as.numeric(unlist(lapply(a$Var1, function(x) { unlist(strsplit(x, "_"))[2] })))
# a$pos2 = as.numeric(unlist(lapply(a$Var2, function(x) { unlist(strsplit(x, "_"))[2] })))
# a$dist = abs(a$pos2 - a$pos1)
# summary(a$dist)



In [9]:
# for (row in c(1:nrow(pairs)))
# {
#     tiss1 = pairs[row,]$tiss1
#     tiss2 = pairs[row,]$tiss2
#     chr = pairs[row,]$chr
    
# #     kb = "200000"
# #     r2 = 0.6
#     r2 = 0.1
#     kb = 500000
    
#     if (tiss1 == "Pancreas")
#     {
#         cmd = paste0("\"", paste("Rscript scripts/17.eQTLFuncEnr/combine_ld.R --tiss1", paste0("\"", tiss1, "\""), "--tiss2", paste0("\"", tiss2, "\""), 
#                 "--chr", gsub("chr", "", chr), "--r2", r2, "--kb",  kb), "\"")
    
#         qsub = paste("echo", cmd, "|", "qsub -N", paste(tiss1, tiss2, gsub("chr", "", chr), sep = "."), 
#                      "-o", "logs/LD_filter.out", "-e", "logs/LD_filter.err", "-V -cwd", "-pe smp 4")
#         message(qsub)
#     #     system(qsub)
#     }
    
    
# }

In [12]:
# r2 = 0.1
# kb = 500000

# files = list.files("pipeline/17.eQTLFuncEnr/ld_all/out")
# # files = files[which(files %like% paste(r2, kb, "filt", sep = "_"))]
# files = files[which(files %like% "0.1")]

# ld_data = lapply(c(1:22), function(chr)
# {
#     message(chr)
#     as.data.frame(rbindlist(lapply(files[which(files %like% paste0("chr", chr, "[.]"))], function(f)
#     {
#         message(f)
#         fread(paste("pipeline/17.eQTLFuncEnr/ld_all/out", f, sep = "/"), data.table = F)
#     }))) %>% distinct()
# })

# names(ld_data) = paste0("chr", c(1:22))

# outfile = paste("pipeline/17.eQTLFuncEnr/ld_all/summary", paste("LD_data", r2, kb, "robj", sep = "."), sep = "/")
# save(ld_data, file = outfile)
# message(paste("Saved:", outfile))

In [13]:
geneinfo = fread("/reference/private/Gencode.v34lift37/gene_info.txt", data.table = F)  %>% 
    mutate(transcript_id = gene_id) %>% select(gene_id, transcript_id, chrom, start, end)

isofinfo = fread("/reference/private/Gencode.v34lift37/isoform_info.txt", data.table = F) %>% 
    select(gene_id, transcript_id, chrom, start, end)

geneinfo = rbind(geneinfo, isofinfo) %>% mutate(transcript_id = simplify_id(transcript_id))

In [14]:
load("manuscript/robj/egene_list.robj", verbose = T)
names(egene_list)

Loading objects:
  egene_list


[1] "gene"    "isoform"

In [ ]:
length(unique(egene_list[[analysis]][[tiss]]$qtl_id))

In [15]:
# r2 = 0.1
# kb = "500000"
# for (analysis in c("gene", "isoform")[1])
# {
#     for (tiss in c("iPSC-PPC", "Pancreas", "Islets")[1])
#     {
#         cmd = paste("qsub -N", paste(analysis, tiss, sep = "."), 
#                     "-V -cwd", "-t", paste0("1-", 10, ":1"), "-tc", "10",
# #                     "-l hostname=\"fl-n-1-17|fl-n-1-3|fl-n-1-13\"", 
#                     "-pe smp 4", 
#                     "-o", paste("logs", paste("LD", tiss, analysis, "out", sep = "."), sep = "/"),
#                     "-e", paste("logs", paste("LD", tiss, analysis, "err", sep = "."), sep = "/"),
#                     "scripts/17.eQTLFuncEnr/ld_by_egene.sh", analysis, tiss, r2, kb)
        
#         message(cmd)
#         system(cmd)
#     }
# }


In [16]:
# get_ld_results = function(qtlid, analysis, tiss1)
# {
#     genes2test = get_genes2test(qtlid, analysis, tiss1)
#     names(genes2test)
    
#     outlist = list()

#     for (tiss2 in c("iPSC-PPC", "Islets", "Pancreas"))
#     {
#         if (tiss2 %in% names(genes2test))
#         {
#             if (tiss2 == "iPSC-PPC")
#             {
#                 genes = all_cred %>% filter(tiss == "iPSC-PPC" & gene_id %in% genes2test[["iPSC-PPC"]]) %>% pull(qtl_id)

#             } else
#             {
#                 genes = paste(tiss2, genes2test[[tiss2]])
#             }

#             snps1 = all_cred %>% filter(qtl_id == qtlid)
#             snps2 = all_cred %>% filter(qtl_id %in% genes)

#             chr = paste0("chr", unlist(strsplit(snps1$snp[1], "_"))[1])

#             t = ld_data[[chr]] %>% filter(Var1 %in% snps1$snp & Var2 %in% snps2$snp)

#             if (nrow(t))
#             {
#                 outlist[[tiss2]] = F
#             } else
#             {
#                 outlist[[tiss2]] = T
#             }
#         } else
#         {
#             outlist[[tiss2]] = F
#         }
#     }
    
#     return(outlist)
# }
    

In [17]:
# get_genes2test = function(qtlid, analysis, tissue)
# {
#     transcript = unlist(strsplit(qtlid, " "))[2]

#     if (analysis == "isoform")
#     {
#         if (tissue == "Islets")
#         {
#             gene = simplify_id(transcript)
#         } else if (tissue == "Pancreas")
#         {
#             gene = simplify_id(unlist(strsplit(transcript, ":"))[5])
#         } else
#         {
#             gene = transcript
#         }
#     } else
#     {
#         gene = transcript
#     }
    
#     message(paste("gene:", gene))

#     chr       = geneinfo[geneinfo$transcript_id == gene,]$chrom
#     start_pos = geneinfo[geneinfo$transcript_id == gene,]$start
#     end_pos   = geneinfo[geneinfo$transcript_id == gene,]$end
    
#     message(paste(chr, start_pos, end_pos))

#     genes2test = geneinfo[geneinfo$chrom == chr & geneinfo$start >= (start_pos - 1e6) & geneinfo$end <= (end_pos + 1e6),]

#     # Get genes in each tissue
#     tiss2test = list()

#     for (g in simplify_id(genes2test$gene_id))
#     { 
#         tiss2test[["Islets"  ]] = c(tiss2test[["Islets"  ]], egene_list[[analysis]][["Islets"  ]] %>% filter(transcript_id %like% g) %>% pull(transcript_id)) 
#         tiss2test[["Pancreas"]] = c(tiss2test[["Pancreas"]], egene_list[[analysis]][["Pancreas"]] %>% filter(transcript_id %like% g) %>% pull(transcript_id)) 
#         tiss2test[["iPSC-PPC"]] = c(tiss2test[["iPSC-PPC"]], egene_list[[analysis]][["iPSC-PPC"]] %>% filter(gene_id %like% g) %>% pull(transcript_id)) 
#     }

#     return(tiss2test)
# }

# Summarize results

In [18]:
# files = list.files("pipeline/17.eQTLFuncEnr/ld_all/processed_0.6_150000")
# length(unique(files))

In [19]:
# out = as.data.frame(rbindlist(lapply(files, function(x)
# {
#     tiss = unlist(strsplit(x, "[.]"))[1]
#     analysis = unlist(strsplit(x, "[.]"))[2]
#     if (tiss == "Islets" & analysis == "isoform")
#     {
#         qtlid = paste("Islets", gsub("Islets.isoform.Islets-", "", gsub(".robj", "", x)))
        
#     }  else if (tiss == "Pancreas" & analysis == "isoform")
#     {
#         qtlid = paste("Pancreas", gsub("Pancreas.isoform.Pancreas-", "", gsub(".robj", "", x)))
#     } else
#     {
#        qtlid = gsub("-", " ", unlist(strsplit(x, "[.]"))[3]) 
#     }

#     load(paste("pipeline/17.eQTLFuncEnr/ld_all/processed_0.6_150000", x, sep = "/"))
    
#     data.frame(tissue = tiss, analysis = analysis, qtl_id = qtlid, in_ld_ppc = outlist[["iPSC-PPC"]], in_ld_islet = outlist[["Islets"]], in_ld_panc = outlist[["Pancreas"]])
# })))

In [20]:
# a = egene_list[["gene"]][["Islets"]] %>% filter(ChrPheno == 23) %>% pull(qtl_id)
# a = c(a, egene_list[["isoform"]][["Islets"]] %>% filter(ChrPheno == 23) %>% pull(qtl_id))
# out = out %>% filter(!qtl_id %in% a)

# out = out %>% filter(in_ld_islet != "NA")

In [21]:
# fwrite(out, "pipeline/17.eQTLFuncEnr/ld_all/summary/LD_results.0.6_150000.txt", row.names = F, sep = "\t")

# Using Lead Putative Causal SNP

In [22]:
load("manuscript/robj/gt1_list.robj", verbose = T)

names(gt1_list[["gene"]])
names(gt1_list[["isoform"]])

gt1_list[[1]][[2]]$snp = paste("VAR", gt1_list[[1]][[2]]$snp, sep = "_")
gt1_list[[2]][[2]]$snp = paste("VAR", gt1_list[[2]][[2]]$snp, sep = "_")

Loading objects:
  gt1_list


[1] "iPSC-PPC" "Islets"   "Pancreas"

[1] "iPSC-PPC" "Islets"   "Pancreas"

In [23]:
for (pheno in names(gt1_list))
{
    for (tiss in names(gt1_list[[pheno]]))
    {
        message(paste(pheno, tiss, length(unique(gt1_list[[pheno]][[tiss]]$qtl_id) )))
    }
}

gene iPSC-PPC 4433

gene Islets 4208

gene Pancreas 9426

isoform iPSC-PPC 4232

isoform Islets 5292

isoform Pancreas 2249



In [24]:
egenes_gt1 = unlist(lapply(names(gt1_list), function(pheno)
{
    unlist(lapply(names(gt1_list[[pheno]]), function(tiss)
    {
        gt1_list[[pheno]][[tiss]]$qtl_id
    }))
}))

qtl2class = fread("pipeline/17.eQTLFuncEnr/ld_all/topsnp/qtl2class.txt", data.table = F)
table(qtl2class$has_gt1)
qtl2class$has_gt1 = ifelse(qtl2class$eqtl_id %in% egenes_gt1, T, F)
table(qtl2class$has_gt1)


 TRUE 
27384 


 TRUE 
27384 

In [492]:
# load("manuscript/robj/99CredibleSet_list.robj", verbose = T)

In [51]:
load("manuscript/robj/egene_list.robj", verbose = T)

all_egenes = unlist(lapply(names(egene_list), function(pheno) 
{
    unlist(lapply(names(egene_list[[pheno]]), function(tiss)
    {
        egene_list[[pheno]][[tiss]]$qtl_id
    }))
}))

head(all_egenes[which(!all_egenes %in% topsnp$qtl_id)])

Loading objects:
  egene_list


[1] "0 ENSG00000004478" "0 ENSG00000005882" "0 ENSG00000006634"
[4] "0 ENSG00000010803" "0 ENSG00000015676" "0 ENSG00000021645"

In [399]:
# ipscore_list = list()

# for (i in c(1:22))
# {
#     message(i)
#     ipscore_list[[paste0("chr", i)]] = paste("VAR", readLines(paste("/frazer01/home/jennifer/references/ipscore/rsid", paste(paste0("chr", i), "snp", sep = "."), sep = "/")), sep = "_")
# }

In [13]:
library(tidyr)

In [45]:
rsid_list = list()

for (i in c(1:22))
{
    message(i)
    this = paste("VAR", readLines(paste("/frazer01/home/jennifer/references/1kg/vcf", paste(paste0("chr", i), "snp", sep = "."), sep = "/")), sep = "_")
    this = data.frame(snp = this)
    this[,c("var", "chr", "pos", "ref", "alt")] = str_split_fixed(this$snp, "_", 5)
    this$chr_pos = paste(this$chr, this$pos, sep = "_")
    rsid_list[[paste0("chr", i)]] = this
}

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22



In [46]:
ngt1_list = gt1_list

for (pheno in names(ngt1_list))
{
    for (tiss in names(ngt1_list[[pheno]]))
    {
        if (tiss != "Pancreas other")
        {
            message(paste(pheno, tiss))
            ngt1_list[[pheno]][[tiss]]$snp = gsub("chr", "VAR_", ngt1_list[[pheno]][[tiss]]$snp)
            ngt1_list[[pheno]][[tiss]][,c("var", "chr", "pos", "ref", "alt")] = str_split_fixed(ngt1_list[[pheno]][[tiss]]$snp, "_", 5)
            ngt1_list[[pheno]][[tiss]]$chr_pos = paste(ngt1_list[[pheno]][[tiss]]$chr, ngt1_list[[pheno]][[tiss]]$pos, sep = "_")
            
            ngt1_list[[pheno]][[tiss]] = as.data.frame(rbindlist(lapply(c(1:22), function(x)
            {
                this = ngt1_list[[pheno]][[tiss]] %>% filter(chr == x)
                if (nrow(this) != 0)
                {
                    this$in_1kg = ifelse(this$chr_pos %in% rsid_list[[paste0("chr", x)]]$chr_pos, T, F)
                    this = merge(this, rsid_list[[paste0("chr", x)]] %>% 
                                     select(chr_pos, snp, ref, alt) %>% 
                                     rename(snp_1kg = snp, ref_1kg = ref, alt_1kg = alt),
                                 all.x = T, by = "chr_pos")
                    return(this)
                }
            })))
            print(table(ngt1_list[[pheno]][[tiss]]$in_1kg))
        }
    }
}

gene iPSC-PPC




 FALSE   TRUE 
 18559 348724 


gene Islets




 FALSE   TRUE 
  1853 233891 


gene Pancreas



...


 FALSE   TRUE 
 18089 309942 


isoform Islets




 FALSE   TRUE 
  3086 348023 


isoform Pancreas




FALSE  TRUE 
 2431 88472 


In [47]:
egenes_gt1 = unlist(lapply(names(ngt1_list), function(pheno)
{
    unlist(lapply(names(ngt1_list[[pheno]]), function(tiss)
    {
        ngt1_list[[pheno]][[tiss]]$qtl_id
    }))
}))

qtl2class = fread("pipeline/17.eQTLFuncEnr/ld_all/topsnp/qtl2class.txt", data.table = F)
table(qtl2class$has_gt1)
qtl2class$has_gt1 = ifelse(qtl2class$eqtl_id %in% egenes_gt1, T, F)
table(qtl2class$has_gt1)


 TRUE 
27384 


 TRUE 
27384 

In [481]:
# library(stringr)
# for (name in names(cred99_list)[1:6])
# {
#     cred99_list[[name]][c("var", "chr", "pos", "ref", "alt")] <- str_split_fixed(cred99_list[[name]]$snp, '_', 5)
#     cred99_list[[name]]$chr_pos = paste(cred99_list[[name]]$chr, cred99_list[[name]]$pos, sep = "_")
#     cred99_list[[name]] = as.data.frame(rbindlist(lapply(c(1:22), function(x)
#     {
#         this = cred99_list[[name]] %>% filter(chr == x)
#         this$in_1kg = ifelse(this$chr_pos %in% rsid_list[[paste0("chr", x)]]$chr_pos, T, F)
#         return(this)
#     })))
#     print(table(cred99_list[[name]]$in_1kg))
# }

In [48]:
for (pheno in names(ngt1_list))
{
    for (tiss in names(ngt1_list[[pheno]]))
    {
        this = ngt1_list[[pheno]][[tiss]]
        this$allele_match = ifelse( (this$ref == this$ref_1kg & this$alt == this$alt_1kg) | 
                                    (this$ref == this$alt_1kg & this$alt == this$ref_1kg) , T, F)
        ngt1_list[[pheno]][[tiss]] = this
    }
}

In [52]:
topsnp = as.data.frame(rbindlist(lapply(names(ngt1_list), function(pheno)
{
    as.data.frame(rbindlist(lapply(names(ngt1_list[[pheno]]), function(x)
    {
        ngt1_list[[pheno]][[x]] %>% filter(snp != "null" & !snp %like% "NA" & in_1kg == T & SNP.PP >= 0.01) %>% 
            select(snp, SNP.PP, qtl_id, pvalues., in_1kg, chr, chr_pos, allele_match, snp_1kg) %>% 
            group_by(qtl_id) %>% filter(SNP.PP == max(SNP.PP)) %>% arrange(desc(allele_match)) %>% filter(duplicated(qtl_id) == F)
    })))
})))

length(unique(all_egenes[which(!all_egenes %in% topsnp$qtl_id)]) )
head(unique(all_egenes[which(!all_egenes %in% topsnp$qtl_id)]))

topsnp$pos = as.numeric(unlist(lapply(topsnp$snp, function(x) { unlist(strsplit(x, "_"))[3] })))
topsnp$transcript_id = unlist(lapply(topsnp$qtl_id, function(x) { unlist(strsplit(x, " "))[2] }))
topsnp$snp = gsub("VAR_", "", topsnp$snp)
topsnp$chr = unlist(lapply(topsnp$snp, function(x) { unlist(strsplit(x, "_"))[1] }))

topsnp$phenotype = ifelse(topsnp$qtl_id %like% "ENST", "alternative_splicing", "gene_expression")
topsnp$phenotype = ifelse(topsnp$qtl_id %like% "_", "alternative_splicing", topsnp$phenotype)

# fwrite(topsnp, "pipeline/17.eQTLFuncEnr/ld_all/topsnp/topsnps.txt", row.names = F, sep = "\t")

[1] 2276

[1] "0 ENSG00000004478" "0 ENSG00000005882" "0 ENSG00000006634"
[4] "0 ENSG00000010803" "0 ENSG00000015676" "0 ENSG00000021645"

In [53]:
egenes_gt1 = as.data.frame(rbindlist(lapply(names(ngt1_list), function(pheno)
{
    as.data.frame(rbindlist(lapply(names(ngt1_list[[pheno]]), function(tiss)
    {
        ngt1_list[[pheno]][[tiss]] %>% filter(SNP.PP >= 0.01) %>% select(in_1kg, SNP.PP, qtl_id)
    })))
})))

qtl2class = fread("pipeline/17.eQTLFuncEnr/ld_all/topsnp/qtl2class.txt", data.table = F)
qtl2class$has_gt1 = ifelse(qtl2class$eqtl_id %in% egenes_gt1$qtl_id, T, F)
qtl2class$in_1kg = ifelse(qtl2class$eqtl_id %in% egenes_gt1[egenes_gt1$in_1kg == T,]$qtl_id, T, F)
qtl2class$in_topsnp = ifelse(qtl2class$eqtl_id %in% topsnp$qtl_id, T, F)

table(qtl2class$has_gt1, qtl2class$in_1kg)
table(qtl2class$has_gt1, qtl2class$in_topsnp)

      
       FALSE  TRUE
  TRUE   436 26948

      
       FALSE  TRUE
  TRUE   436 26948

In [538]:
# topsnp = as.data.frame(rbindlist(lapply(names(cred99_list)[1:6], function(x)
# {
#     cred99_list[[x]] %>% filter(snp != "null" & !snp %like% "NA" & in_1kg == T) %>% 
#         select(snp, SNP.PP, qtl_id, pvalues., in_1kg) %>% group_by(qtl_id) %>% filter(SNP.PP == max(SNP.PP)) 
# })))

# table(topsnp$in_1kg)

# length(unique(all_egenes[which(!all_egenes %in% topsnp[topsnp$SNP.PP >= 0.01,]$qtl_id)]) )
# head(unique(all_egenes[which(!all_egenes %in% topsnp[topsnp$SNP.PP >= 0.01,]$qtl_id)]))

In [699]:
head(topsnp)

,snp,SNP.PP,qtl_id,pvalues.,in_1kg,chr,chr_pos,allele_match,snp_1kg,pos,transcript_id,phenotype
,<chr>,<dbl>,<chr>,<dbl>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>
1,1_100012794_A_G,0.08856656,0 ENSG00000099260,5.329953e-16,TRUE,1,1_100012794,TRUE,VAR_1_100012794_A_G,100012794,ENSG00000099260,gene_expression
2,1_100181897_G_A,0.03000826,0 ENSG00000156869,2.017248e-09,TRUE,1,1_100181897,TRUE,VAR_1_100181897_G_A,100181897,ENSG00000156869,gene_expression
3,1_100807723_C_CG,0.01441180,1 ENSG00000224616,5.031969e-05,TRUE,1,1_100807723,TRUE,VAR_1_100807723_C_CG,100807723,ENSG00000224616,gene_expression
4,1_100824378_T_C,0.02348328,0 ENSG00000162688,6.275401e-07,TRUE,1,1_100824378,TRUE,VAR_1_100824378_T_C,100824378,ENSG00000162688,gene_expression
5,1_100930482_CTT_C,0.01003377,0 ENSG00000224616,6.363798e-07,TRUE,1,1_100930482,TRUE,VAR_1_100930482_CTT_C,100930482,ENSG00000224616,gene_expression
6,1_101653776_C_T,0.01796027,0 ENSG00000273204,7.267889e-08,TRUE,1,1_101653776,TRUE,VAR_1_101653776_C_T,101653776,ENSG00000273204,gene_expression


In [54]:
topsnp$snp = gsub("VAR_", "", topsnp$snp)
topsnp$snp = paste("VAR", topsnp$snp, sep = "_")

for (i in c(1:22))
{
    this = unique(topsnp$snp_1kg[which(topsnp$snp_1kg %like% paste0("VAR_", i, "_"))])
    message(paste(i, length(this)))
#     writeLines(gsub("VAR_", "", this), paste("pipeline/17.eQTLFuncEnr/ld_all/topsnp/input", paste0("chr", i, ".txt"), sep = "/"), sep = "\n")
}

1 2216

2 1659

3 1404

4 949

5 1151

6 1423

7 1368

8 811

9 969

10 999

11 1384

12 1116

13 418

14 793

15 808

16 1100

17 1373

18 331

19 1735

20 613

21 338

22 703



In [877]:
# run plink

for (c in c(1:22))
{
    outfile = paste("pipeline/17.eQTLFuncEnr/ld_all/topsnp/output", paste0("chr", c), sep = "/")
    snpfile = paste("pipeline/17.eQTLFuncEnr/ld_all/topsnp/input", paste0("chr", c, ".txt"), sep = "/")

    cmd = paste("plink", "--memory 15000 --threads 4", "--vcf", paste0("~/references/1kg/vcf/", paste0("chr", c), ".vcf.gz"), 
          "--extract", snpfile, "--keep-allele-order", "--r2", "square", "--make-bed", "--out", outfile)

    qsub_cmd = paste("echo", paste0("\"", cmd, "\"" ), "|", "qsub -N", paste0("chr", c),
                     "-pe smp 4", "-V -cwd", "-o", paste("logs", paste(paste0("chr", c), "out", sep = "."), sep = "/"),
#                      "-l hostname=\"fl-n-1-17|fl-n-1-13|fl-n-1-9\"", 
                     "-e", paste("logs", paste(paste0("chr", c), "err", sep = "."), sep = "/"))
    message(qsub_cmd)
    system(qsub_cmd)
}


echo "plink --memory 15000 --threads 4 --vcf ~/references/1kg/vcf/chr1.vcf.gz --extract pipeline/17.eQTLFuncEnr/ld_all/topsnp/input/chr1.txt --keep-allele-order --r2 square --make-bed --out pipeline/17.eQTLFuncEnr/ld_all/topsnp/output/chr1" | qsub -N chr1 -pe smp 4 -V -cwd -o logs/chr1.out -e logs/chr1.err

echo "plink --memory 15000 --threads 4 --vcf ~/references/1kg/vcf/chr2.vcf.gz --extract pipeline/17.eQTLFuncEnr/ld_all/topsnp/input/chr2.txt --keep-allele-order --r2 square --make-bed --out pipeline/17.eQTLFuncEnr/ld_all/topsnp/output/chr2" | qsub -N chr2 -pe smp 4 -V -cwd -o logs/chr2.out -e logs/chr2.err

echo "plink --memory 15000 --threads 4 --vcf ~/references/1kg/vcf/chr3.vcf.gz --extract pipeline/17.eQTLFuncEnr/ld_all/topsnp/input/chr3.txt --keep-allele-order --r2 square --make-bed --out pipeline/17.eQTLFuncEnr/ld_all/topsnp/output/chr3" | qsub -N chr3 -pe smp 4 -V -cwd -o logs/chr3.out -e logs/chr3.err

echo "plink --memory 15000 --threads 4 --vcf ~/references/1kg/vcf/chr4.vc

In [878]:
ld_list = list()

for (x in c(1:22))
{
    message(x)
    ld = fread(paste("pipeline/17.eQTLFuncEnr/ld_all/topsnp/output", paste(paste0("chr", x), "ld", sep = "."), sep = "/"), data.table = F)
    bim = fread(paste("pipeline/17.eQTLFuncEnr/ld_all/topsnp/output", paste(paste0("chr", x), "bim", sep = "."), sep = "/"), data.table = F)
    rownames(ld) = bim$V2
    colnames(ld) = bim$V2
    ld = suppressWarnings(melt(as.matrix(ld))) %>% mutate(Var1 = as.vector(Var1), Var2 = as.vector(Var2))
    ld_list[[paste0("chr", x)]] = list("ld" = ld, "bim" = bim)
}

save(ld_list, file = "pipeline/17.eQTLFuncEnr/ld_all/topsnp/ld_list.robj")

1

2

3

4

5

6

7

8

9

10

11

12

13

14

15

16

17

18

19

20

21

22



In [55]:
load("pipeline/17.eQTLFuncEnr/ld_all/topsnp/ld_list.robj", verbose = T)

Loading objects:
  ld_list


In [56]:
all_topsnp = as.data.frame(rbindlist(lapply(names(ngt1_list), function(pheno)
{
    as.data.frame(rbindlist(lapply(names(ngt1_list[[pheno]]), function(x)
    {
        ngt1_list[[pheno]][[x]] %>% filter(snp != "null" & !snp %like% "NA" & SNP.PP >= 0.01) %>% 
            select(snp, SNP.PP, qtl_id, pvalues., in_1kg, chr, chr_pos, allele_match, snp_1kg) %>%
            filter(!qtl_id %in% topsnp$qtl_id) %>% group_by(qtl_id) %>% 
            filter(SNP.PP == max(SNP.PP)) %>% arrange(desc(allele_match)) %>% filter(duplicated(qtl_id) == F)
    })))
})))

all_topsnp$pos = as.numeric(unlist(lapply(all_topsnp$snp, function(x) { unlist(strsplit(x, "_"))[3] })))
all_topsnp$transcript_id = unlist(lapply(all_topsnp$qtl_id, function(x) { unlist(strsplit(x, " "))[2] }))
all_topsnp$snp = gsub("VAR_", "", all_topsnp$snp)
all_topsnp$chr = unlist(lapply(all_topsnp$snp, function(x) { unlist(strsplit(x, "_"))[1] }))

all_topsnp$phenotype = ifelse(all_topsnp$qtl_id %like% "ENST", "alternative_splicing", "gene_expression")
all_topsnp$phenotype = ifelse(all_topsnp$qtl_id %like% "_", "alternative_splicing", all_topsnp$phenotype)
all_topsnp = rbind(all_topsnp, topsnp)

# Processed V2

In [57]:
egenes_gt1 = as.data.frame(rbindlist(lapply(names(ngt1_list), function(pheno)
{
    as.data.frame(rbindlist(lapply(names(ngt1_list[[pheno]]), function(tiss)
    {
        ngt1_list[[pheno]][[tiss]] %>% filter(SNP.PP >= 0.01) %>% select(in_1kg, SNP.PP, qtl_id)
    })))
})))

qtl2class = fread("pipeline/17.eQTLFuncEnr/ld_all/topsnp/qtl2class.txt", data.table = F)
qtl2class$has_gt1 = ifelse(qtl2class$eqtl_id %in% egenes_gt1$qtl_id, T, F)
qtl2class$in_1kg = ifelse(qtl2class$eqtl_id %in% egenes_gt1[egenes_gt1$in_1kg == T,]$qtl_id, T, F)
qtl2class$in_topsnp = ifelse(qtl2class$eqtl_id %in% topsnp$qtl_id, T, F)
qtl2class$in_alltopsnp = ifelse(qtl2class$eqtl_id %in% all_topsnp$qtl_id, T, F)
qtl2class = merge(qtl2class, all_topsnp[, c("qtl_id", "chr", "pos", "snp_1kg")] %>% rename(eqtl_id = qtl_id), by = "eqtl_id", all.x = T)

table(qtl2class$has_gt1, qtl2class$in_1kg)
table(qtl2class$has_gt1, qtl2class$in_topsnp)
table(qtl2class$has_gt1, qtl2class$in_alltopsnp)



      
       FALSE  TRUE
  TRUE   436 26948

      
       FALSE  TRUE
  TRUE   436 26948

      
        TRUE
  TRUE 27384

In [58]:
qtl2class = as.data.frame(rbindlist(lapply(c(1:22), function(x)
{
    this = qtl2class %>% filter(chr == x)
    this$ld_tested = ifelse(gsub("VAR_", "", this$snp_1kg) %in% ld_list[[paste0("chr", x)]]$bim$V2, T, F)
    return(this)
})))

table(qtl2class$ld_tested)


FALSE  TRUE 
  436 26948 

In [59]:
qtl2class$eqtl_type = ifelse(qtl2class$module_id %like% "GE" | qtl2class$module_id %like% "AS", "module", "singleton")

In [979]:
fwrite(qtl2class, "pipeline/17.eQTLFuncEnr/ld_all/topsnp/qtl2class_annotated.txt", row.names = F, sep = "\t")

In [982]:
kb = 500
r2 = 0.2

ldstats = as.data.frame(rbindlist(lapply(unique(qtl2class$eqtl_id)[1], function(id)
{
    transcript = unlist(strsplit(id, " "))[2]
    pheno = qtl2class[qtl2class$eqtl_id == id,]$eqtl_phenotype

    this = qtl2class %>% filter(eqtl_id == id)
    snpchr = this$chr
    this_tiss = this$tissue
    eqtltype = this$eqtl_type
    mod_id = this$module_id
    
    message(paste(transcript, pheno, snpchr, this_tiss, eqtltype, mod_id))

    # get all other eQTLs
    work = qtl2class %>% filter(eqtl_id != id & chr == snpchr & eqtl_phenotype == pheno & module_id != mod_id) %>% mutate(dist = abs(pos - this$pos))

    # get all snps for ld
    for (tiss in unique(work$tissue))
    {
        # get snps for tissue
        if (this_tiss == tiss)
        {
            snps_tiss = work %>% filter(tissue == tiss & chr == snpchr & transcript_id != transcript_id) %>% mutate(snp_1kg = gsub("VAR_", "", snp_1kg)) 
        } else
        {
            snps_tiss = work %>% filter(tissue == tiss & chr == snpchr) %>% mutate(snp_1kg = gsub("VAR_", "", snp_1kg)) 
        }

        # get ld results for snps
        ld_tiss = ld_list[[paste0("chr", snpchr)]]$ld %>% filter(Var1 == gsub("VAR_", "", this$snp_1kg) & Var2 %in% gsub("VAR_", "", snps_tiss$snp_1kg))
        
        # merge ld with snps
        snps_tiss = merge(snps_tiss, ld_tiss %>% select(Var2, value) %>% rename(snp_1kg = Var2, ld = value), by = "snp_1kg", all.x = T) 
        
        # if snp is outside window, not in LD, else, test for LD. if can't test for LD, then in LD, else, if ld > 0.6, then in LD, else not in LD
        snps_tiss$in_ld = ifelse(snps_tiss$dist >= kb*1e3, F, ifelse(is.na(snps_tiss$snp_1kg), T, ifelse(snps_tiss$ld >= r2, T, F)))
        #snps_tiss$in_ld = ifelse(is.na(snps_tiss$snp_1kg) & snps_tiss$dist >= kb*1e3, F, ifelse(snps_tiss$dist >= kb*1e3 & snps_tiss$ld <= r2, F, T))
        
        # if one eQTL in LD, then yes, else no
        this[,paste0("ld_", tiss)] = ifelse(nrow(snps_tiss[snps_tiss$in_ld == T,]) != 0, T, F)
    }

    return(this)
})))

ldstats

ENSG00000007341 gene_expression 1 ipsc_ppc module GE_1_10



eqtl_id,tissue,eqtl_phenotype,transcript_id,gene_id,in_gencode,in_network,expressed_fetal,expressed_endocrine,expressed_exocrine,⋯,in_topsnp,in_alltopsnp,chr,pos,snp_1kg,ld_tested,eqtl_type,ld_ipsc_ppc,ld_endocrine,ld_exocrine
<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,⋯,<lgl>,<lgl>,<chr>,<dbl>,<chr>,<lgl>,<chr>,<lgl>,<lgl>,<lgl>
0 ENSG00000007341,ipsc_ppc,gene_expression,ENSG00000007341,ENSG00000007341,TRUE,TRUE,TRUE,TRUE,TRUE,⋯,TRUE,TRUE,1,113061959,VAR_1_113061959_G_C,TRUE,module,FALSE,FALSE,FALSE


In [63]:
id = "0 ENSG00000100380"
kb = 500
r2 = 0.2
chr = 22

transcript = unlist(strsplit(id, " "))[2]
pheno = qtl2class[qtl2class$eqtl_id == id,]$eqtl_phenotype

this = qtl2class %>% filter(eqtl_id == id)
snpchr = this$chr
snpchr = 22
this_tiss = this$tissue
eqtltype = this$eqtl_type
mod_id = this$module_id

message(mod_id)
message(paste(transcript, pheno, snpchr, this_tiss, mod_id))

# get all other eQTLs
work = qtl2class %>% filter(eqtl_id != id & chr == snpchr & eqtl_phenotype == pheno & module_id != mod_id) %>% mutate(dist = abs(pos - this$pos))


0 ENSG00000100380

ENSG00000100380 gene_expression 22 ipsc_ppc 0 ENSG00000100380



In [65]:
# get all snps for ld
for (tiss in unique(work$tissue))
{
    # get snps for tissue
    if (this_tiss == tiss)
    {
        snps_tiss = work %>% filter(tissue == tiss & chr == snpchr & transcript_id != transcript) %>% mutate(snp_1kg = gsub("VAR_", "", snp_1kg)) 
    } else
    {
        snps_tiss = work %>% filter(tissue == tiss & chr == snpchr) %>% mutate(snp_1kg = gsub("VAR_", "", snp_1kg)) 
    }
    
    # get ld results for snps
    ld_tiss = ld_list[[paste0("chr", snpchr)]]$ld %>% filter(Var1 == gsub("VAR_", "", this$snp_1kg) & Var2 %in% gsub("VAR_", "", snps_tiss$snp_1kg))
    
    # merge ld with snps
    snps_tiss = merge(snps_tiss, ld_tiss %>% select(Var2, value) %>% rename(snp_1kg = Var2, ld = value), by = "snp_1kg", all.x = T) 

    # if snp is outside window, not in LD, else, test for LD. if can't test for LD, then in LD, else, if ld > 0.6, then in LD, else not in LD
    snps_tiss$in_ld = ifelse(snps_tiss$dist >= kb*1e3, F, ifelse(is.na(snps_tiss$snp_1kg), T, ifelse(snps_tiss$ld >= r2, T, F)))
    #snps_tiss$in_ld = ifelse(is.na(snps_tiss$snp_1kg) & snps_tiss$dist >= kb*1e3, F, ifelse(snps_tiss$dist >= kb*1e3 & snps_tiss$ld <= r2, F, T))

    # if one eQTL in LD, then yes, else no
    this[,paste0("ld_", tiss)] = ifelse(nrow(snps_tiss[snps_tiss$in_ld == T,]) != 0, T, F)
}

str(this)



'data.frame':	1 obs. of  24 variables:
 $ eqtl_id            : chr "0 ENSG00000100380"
 $ tissue             : chr "ipsc_ppc"
 $ eqtl_phenotype     : chr "gene_expression"
 $ transcript_id      : chr "ENSG00000100380"
 $ gene_id            : chr "ENSG00000100380"
 $ in_gencode         : logi TRUE
 $ in_network         : logi FALSE
 $ expressed_fetal    : logi TRUE
 $ expressed_endocrine: logi TRUE
 $ expressed_exocrine : logi TRUE
 $ has_gt1            : logi TRUE
 $ in_1kg             : logi TRUE
 $ gene_name          : chr "ST13"
 $ module_id          : chr "0 ENSG00000100380"
 $ in_topsnp          : logi TRUE
 $ in_alltopsnp       : logi TRUE
 $ chr                : chr "22"
 $ pos                : num 41332598
 $ snp_1kg            : chr "VAR_22_41332598_A_AT"
 $ ld_tested          : logi TRUE
 $ eqtl_type          : chr "singleton"
 $ ld_ipsc_ppc        : logi FALSE
 $ ld_endocrine       : logi FALSE
 $ ld_exocrine        : logi FALSE


In [67]:
41332598 > 41049522

41332598 < 41449522

[1] TRUE

[1] TRUE

In [ ]:

# ldstats = as.data.frame(rbindlist(lapply(c(id), function(id)
# {
#     transcript = unlist(strsplit(id, " "))[2]
#     pheno = qtl2class[qtl2class$eqtl_id == id,]$eqtl_phenotype

#     this = qtl2class %>% filter(eqtl_id == id)
#     snpchr = this$chr
#     this_tiss = this$tissue
    
#     message(paste(transcript, pheno, snpchr))

#     work = qtl2class %>% filter(eqtl_id != id & chr == snpchr & eqtl_phenotype == pheno) %>% mutate(dist = abs(pos - this$pos))

#     # get all snps for ld
#     for (tiss in unique(work$tissue))
#     {
#         if (this_tiss == tiss)
#         {
#             snps_tiss = work %>% filter(tissue == tiss & chr == snpchr & transcript_id != transcript_id) %>% mutate(snp_1kg = gsub("VAR_", "", snp_1kg)) 
#         } else
#         {
#             # get snps for tissue
#             snps_tiss = work %>% filter(tissue == tiss & chr == snpchr) %>% mutate(snp_1kg = gsub("VAR_", "", snp_1kg)) 
#         }
        
#         # get ld results for snps
#         ld_tiss = ld_list[[paste0("chr", snpchr)]]$ld %>% filter(Var1 == gsub("VAR_", "", this$snp_1kg) & Var2 %in% gsub("VAR_", "", snps_tiss$snp_1kg))
        
#         # merge ld with snps
#         snps_tiss = merge(snps_tiss, ld_tiss %>% select(Var2, value) %>% rename(snp_1kg = Var2, ld = value), by = "snp_1kg", all.x = T) 
        
#         # if snp is outside window, not in LD, else, test for LD. if can't test for LD, then in LD, else, if ld > 0.6, then in LD, else not in LD
#         snps_tiss$in_ld = ifelse(snps_tiss$dist >= kb*1e3, F, ifelse(is.na(snps_tiss$snp_1kg), T, ifelse(snps_tiss$ld >= r2, T, F)))
        
#         # if one eQTL in LD, then yes, else no
#         this[,paste0("ld_", tiss)] = ifelse(nrow(snps_tiss[snps_tiss$in_ld == T,]) != 0, T, F)
#     }

#     return(this)
# })))

# ldstats

In [1027]:
# 300 0.3
# 300 0.6
# 250 0.6
# 500 0.3

kb = 300
r2 = 0.2

for (i in c(1:22))
{
    chr = i
    
    cmd = paste("echo", "\"Rscript", "scripts/17.eQTLFuncEnr/topsnp_ld2.R", "--chr", chr, "--r2", r2, "--kb", paste0(kb, "\""))
    
    qsub = paste(cmd, "|", "qsub", "-N", paste0("chr", chr), 
                 "-o", paste("logs", paste(paste("topsnp", chr, sep = "_"), "out", sep = "."), sep = "/"),
                 "-e", paste("logs", paste(paste("topsnp", chr, sep = "_"), "err", sep = "."), sep = "/"),
                 "-pe smp 4", "-V -cwd")
    
    message(qsub)
    system(qsub)
    
}

echo "Rscript scripts/17.eQTLFuncEnr/topsnp_ld2.R --chr 1 --r2 0.2 --kb 500" | qsub -N chr1 -o logs/topsnp_1.out -e logs/topsnp_1.err -pe smp 4 -V -cwd

Warning message in system(qsub):
“system call failed: Cannot allocate memory”
Warning message in system(qsub):
“error in running command”
echo "Rscript scripts/17.eQTLFuncEnr/topsnp_ld2.R --chr 2 --r2 0.2 --kb 500" | qsub -N chr2 -o logs/topsnp_2.out -e logs/topsnp_2.err -pe smp 4 -V -cwd

Warning message in system(qsub):
“system call failed: Cannot allocate memory”
Warning message in system(qsub):
“error in running command”
echo "Rscript scripts/17.eQTLFuncEnr/topsnp_ld2.R --chr 3 --r2 0.2 --kb 500" | qsub -N chr3 -o logs/topsnp_3.out -e logs/topsnp_3.err -pe smp 4 -V -cwd

Warning message in system(qsub):
“system call failed: Cannot allocate memory”
Warning message in system(qsub):
“error in running command”
echo "Rscript scripts/17.eQTLFuncEnr/topsnp_ld2.R --chr 4 --r2 0.2 --kb 500" | qsub -N chr4 -o logs/topsnp_4.out -e logs/topsnp

In [927]:
kb = 500
r2 = 0.1
files = list.files("pipeline/17.eQTLFuncEnr/ld_all/topsnp/processed2", full.names = T)
files = files[which(files %like% paste0(kb, "kb") & files %like% paste0("r", r2))]

nqtl2class = as.data.frame(rbindlist(lapply(files, function(x) { fread(x, data.table = F) })))

nqtl2class %>% filter(eqtl_id == "0 ENSG00000196821")



eqtl_id,tissue,eqtl_phenotype,transcript_id,gene_id,in_gencode,in_network,expressed_fetal,expressed_endocrine,expressed_exocrine,⋯,gene_name,in_topsnp,in_alltopsnp,chr,pos,snp_1kg,ld_tested,ld_ipsc_ppc,ld_endocrine,ld_exocrine
<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,⋯,<chr>,<lgl>,<lgl>,<int>,<int>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>
0 ENSG00000196821,ipsc_ppc,gene_expression,ENSG00000196821,ENSG00000196821,TRUE,FALSE,TRUE,TRUE,TRUE,⋯,ILRUN,TRUE,TRUE,6,34658565,VAR_6_34658565_A_AGTTTT,TRUE,TRUE,TRUE,TRUE


# Processed v1

In [162]:
get_stats = function(id, kb)
{
    transcript = unlist(strsplit(id, " "))[2]
    out = data.frame(eqtl_id = id)
    pheno = topsnp[topsnp$qtl_id == id,]$phenotype
    out$phenotype = pheno
    snps_a = topsnp %>% filter(qtl_id == id)
    snps_f = topsnp %>% filter(phenotype == pheno & qtl_id != id & transcript_id != transcript & !qtl_id %like% "Islets" & !qtl_id %like% "Pancreas") %>% mutate(dist = abs(pos - snps_a$pos))
    snps_i = topsnp %>% filter(phenotype == pheno & qtl_id != id & qtl_id %like% "Islets") %>% mutate(dist = abs(pos - snps_a$pos))
    snps_p = topsnp %>% filter(phenotype == pheno & qtl_id != id & qtl_id %like% "Pancreas") %>% mutate(dist = abs(pos - snps_a$pos))

    out$topsnp = snps_a$snp
    # distance to nearest eQTL 
    out$dist_f = min(snps_f$dist)
    out$dist_i = min(snps_i$dist)
    out$dist_p = min(snps_p$dist)

    # max r2
    chr = unlist(strsplit(snps_a$snp, "_"))[1]
    l_f = ld_list[[paste0("chr", chr)]] %>% filter(Var1 == snps_a$snp & Var2 %in% snps_f[snps_f$dist <= kb*1e3,]$snp)
    l_i = ld_list[[paste0("chr", chr)]] %>% filter(Var1 == snps_a$snp & Var2 %in% snps_i[snps_i$dist <= kb*1e3,]$snp)
    l_p = ld_list[[paste0("chr", chr)]] %>% filter(Var1 == snps_a$snp & Var2 %in% snps_p[snps_p$dist <= kb*1e3,]$snp)
    out$r2_f = suppressWarnings(suppressMessages(max(l_f$value)))
    out$r2_i = suppressWarnings(suppressMessages(max(l_i$value)))
    out$r2_p = suppressWarnings(suppressMessages(max(l_p$value)))
    
    out$r2_f = ifelse(is.infinite(out$r2_f), NA, out$r2_f)
    out$r2_i = ifelse(is.infinite(out$r2_i), NA, out$r2_i)
    out$r2_p = ifelse(is.infinite(out$r2_p), NA, out$r2_p)
    
    return(out)
}


In [166]:
ldstats = as.data.frame(rbindlist(lapply(unique(all_egenes)[1:10], function(x) { get_stats(x, 300) } )))
ldstats

eqtl_id,phenotype,topsnp,dist_f,dist_i,dist_p,r2_f,r2_i,r2_p
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0 ENSG00000000419,gene_expression,20_50005043_G_GA,3070,1841,1226,NA,9.27000e-03,0.00420204
0 ENSG00000001561,gene_expression,6_46148778_AT_A,2660,14072,3753,0.11027700,2.14985e-01,0.19410300
0 ENSG00000002726,gene_expression,7_150521096_A_G,4805,12658,8306,0.00864343,5.27728e-02,0.21603100
0 ENSG00000003249,gene_expression,16_90081660_A_G,2242,3479,3479,0.15068300,2.62493e-01,0.53386700
0 ENSG00000003436,gene_expression,2_188783299_G_A,138141,386832,6768,NA,NA,0.63058100
0 ENSG00000003987,gene_expression,8_17267851_T_A,530,8372,3555,0.00250768,4.03714e-03,0.06499440
0 ENSG00000004139,gene_expression,17_26703376_A_G,5994,6894,306,0.84830200,9.88523e-01,0.99569100
0 ENSG00000004478,gene_expression,12_2925496_CT_C,7075,8794,4422,NA,1.29113e-05,0.43572300
0 ENSG00000004534,gene_expression,3_49989935_GT_G,2006,16949,13882,0.44865500,5.87851e-01,0.79947700


In [221]:
for (i in c(1:22))
{
    chr = i
    kb = 250
    
    cmd = paste("echo", "\"Rscript", "scripts/17.eQTLFuncEnr/topsnp_ld.R", "--chr", chr, "--kb", paste0(kb, "\""))
    
    qsub = paste(cmd, "|", "qsub", "-N", paste0("chr", chr), 
                 "-o", paste("logs", paste(paste("topsnp", chr, sep = "_"), "out", sep = "."), sep = "/"),
                 "-e", paste("logs", paste(paste("topsnp", chr, sep = "_"), "err", sep = "."), sep = "/"),
                 "-pe smp 4", "-V -cwd")
    
    message(qsub)
    system(qsub)
    
}

echo "Rscript scripts/17.eQTLFuncEnr/topsnp_ld.R --chr 1 --kb 250" | qsub -N chr1 -o logs/topsnp_1.out -e logs/topsnp_1.err -pe smp 4 -V -cwd

echo "Rscript scripts/17.eQTLFuncEnr/topsnp_ld.R --chr 2 --kb 250" | qsub -N chr2 -o logs/topsnp_2.out -e logs/topsnp_2.err -pe smp 4 -V -cwd

echo "Rscript scripts/17.eQTLFuncEnr/topsnp_ld.R --chr 3 --kb 250" | qsub -N chr3 -o logs/topsnp_3.out -e logs/topsnp_3.err -pe smp 4 -V -cwd

echo "Rscript scripts/17.eQTLFuncEnr/topsnp_ld.R --chr 4 --kb 250" | qsub -N chr4 -o logs/topsnp_4.out -e logs/topsnp_4.err -pe smp 4 -V -cwd

echo "Rscript scripts/17.eQTLFuncEnr/topsnp_ld.R --chr 5 --kb 250" | qsub -N chr5 -o logs/topsnp_5.out -e logs/topsnp_5.err -pe smp 4 -V -cwd

echo "Rscript scripts/17.eQTLFuncEnr/topsnp_ld.R --chr 6 --kb 250" | qsub -N chr6 -o logs/topsnp_6.out -e logs/topsnp_6.err -pe smp 4 -V -cwd

echo "Rscript scripts/17.eQTLFuncEnr/topsnp_ld.R --chr 7 --kb 250" | qsub -N chr7 -o logs/topsnp_7.out -e logs/topsnp_7.err -pe smp 4 -V -cwd


In [248]:
kb = 250
files = list.files("pipeline/17.eQTLFuncEnr/ld_all/topsnp/processed", full.names = T)
files = files[which(files %like% kb)]
ldstats = as.data.frame(rbindlist(lapply(files, function(f)
{
    fread(f, data.table = F)
}))) 

ldstats$chr = unlist(lapply(ldstats$topsnp, function(x) { unlist(strsplit(x, "_"))[1] }))
ldstats = as.data.frame(rbindlist(lapply(unique(ldstats$chr), function(x) 
{
    this = ldstats %>% filter(chr == x)
    this$in_1kg = ifelse(this$topsnp %in% ld_list[[paste0("chr", x)]]$Var1, T, F)
    return(this)
})))

# ldstats$ld_f = ifelse( (ldstats$dist_f >= kb*1e3 & ldstats$r2_f <= 0.6) | 
#                       (ldstats$dist_f >= kb*1e3 & ldstats$in_1kg == T), F, ifelse(ldstats$in_1kg == F, "not_in_1kg", T))

# ldstats$ld_i = ifelse( (ldstats$dist_i >= kb*1e3 & ldstats$r2_i <= 0.6) | 
#                       (ldstats$dist_i >= kb*1e3 & ldstats$in_1kg == T), F, ifelse(ldstats$in_1kg == F, "not_in_1kg", T))

# ldstats$ld_p = ifelse( (ldstats$dist_p >= kb*1e3 & ldstats$r2_p <= 0.6) | 
#                       (ldstats$dist_p >= kb*1e3 & ldstats$in_1kg == T), F, ifelse(ldstats$in_1kg == F, "not_in_1kg", T))

ldstats$ld_f = ifelse( ldstats$r2_f <= 0.6 | (ldstats$dist_f >= kb*1e3 & ldstats$in_1kg == T), F, ifelse(ldstats$in_1kg == F, "not_in_1kg", T))

ldstats$ld_i = ifelse( ldstats$r2_i <= 0.6 | (ldstats$dist_i >= kb*1e3 & ldstats$in_1kg == T), F, ifelse(ldstats$in_1kg == F, "not_in_1kg", T))

ldstats$ld_p = ifelse( ldstats$r2_p <= 0.6 | (ldstats$dist_p >= kb*1e3 & ldstats$in_1kg == T), F, ifelse(ldstats$in_1kg == F, "not_in_1kg", T))

table(ldstats$ld_f)
table(ldstats$ld_i)
table(ldstats$ld_p)


FALSE  TRUE 
27230  4619 


FALSE  TRUE 
27520  5262 


FALSE  TRUE 
26812  5831 

In [249]:
ldstats %>% filter(ld_f == F & ld_i == F & ld_p == F)

eqtl_id,phenotype,topsnp,dist_f,dist_i,dist_p,r2_f,r2_i,r2_p,chr,in_1kg,ld_f,ld_i,ld_p
<chr>,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>
Pancreas ENSG00000173269,gene_expression,10_88718436_A_C,164351,11876,8021,0.14243500,0.279692000,3.86061e-01,10,TRUE,FALSE,FALSE,FALSE
0 ENSG00000148702,gene_expression,10_115248677_G_T,1097551,190717,118706,NA,0.080979600,8.09796e-02,10,TRUE,FALSE,FALSE,FALSE
Islets ENSG00000174194,gene_expression,10_51585752_C_T,28831,41277,13924,0.00953055,0.017568000,1.51773e-03,10,TRUE,FALSE,FALSE,FALSE
1 ENST00000374056,alternative_splicing,10_52018090_T_C,92123,8406382,155913,0.02732000,NA,3.12876e-02,10,TRUE,FALSE,FALSE,FALSE
Pancreas ENSG00000019995,gene_expression,10_125717416_A_G,65516,44558,65515,0.00381160,0.090048100,4.64622e-04,10,TRUE,FALSE,FALSE,FALSE
Pancreas chr10:80176155:80180941:clu_9023:ENSG00000122359.17,alternative_splicing,10_81938096_T_C,52088,3083,124832,0.02175170,0.103982000,2.94096e-02,10,TRUE,FALSE,FALSE,FALSE
0 ENST00000466484,alternative_splicing,10_99056921_C_G,170038,98640,371545,0.05610890,0.072266300,NA,10,TRUE,FALSE,FALSE,FALSE
Pancreas chr10:102727157:102728431:clu_9520:ENSG00000156398.12,alternative_splicing,10_104487382_C_T,31358,1733349,7507,0.07365610,NA,1.17010e-01,10,TRUE,FALSE,FALSE,FALSE
Pancreas ENSG00000072422,gene_expression,10_62259224_G_A,395965,1172824,196538,NA,NA,1.29568e-01,10,TRUE,FALSE,FALSE,FALSE
